# Notebook preparación análisis geográfico comparativo
Este notebook realiza el procesamiento y preparación de datos de la encuesta de las preguntas/variables numéricas de la encuesta de "Prácticas deportivas y calidad de Vida 2024" de la Encuesta Bienal de Culturas de la Dirección Observatorio y Gestión del Conocimiento Cultural.

## Tiene estas características: 
- Realiza la lectura de datos del archivo "m182_datos.xls", que cuenta con una estructura del modelo de analítica de mediciones.
- En la hoja variables identifica qué columnas son numéricas para hacer los resúmenes.
- Calcula los promedios ponderados de cada variable para cada localidad, considerando el factor de expansión (FACTOR).
- Agrega una fila para el total de la ciudad.
- Agrega comparaciones relativas de desviación y 

## Sobre el notebook:
- Preparado por: javier.ojeda@scrd.gov
- Sistemas de Información y Narrativas
- Dirección Observatorio y Gestión del Conocimiento Cultural
- Secretaría de Cultura, Recreación y Deportes | 2025

In [13]:
import pandas as pd

In [14]:
finename = 'm182_datos.xlsx'
df_secciones = pd.read_excel(finename, sheet_name='secciones')
df_preguntas = pd.read_excel(finename, sheet_name='preguntas')
df_variables = pd.read_excel(finename, sheet_name='variables')
df_localidades = pd.read_excel(finename, sheet_name='localidades')
df_respuestas = pd.read_excel(finename, sheet_name='datos')

In [15]:
columna_dimension = 'P0003001000'   # Corresponde a la localidad de la vivienda del encuestado
columna_dimension_nombre = 'localidad'

In [16]:
# Valor inicial para pruebas
variables_numericas = ['P1500033060']

In [17]:
# Se crea el df_respuestas_acumuladas, inicialmente vacío
df_respuestas_acumuladas = pd.DataFrame()

In [18]:
# Se genera la lista de variables_numericas a partir del df_variables, donde la columna unidad_medida tiene un valor con al menos un caracter
variables_numericas = df_variables[df_variables['unidad_medida'].fillna('').str.strip() != '']['codigo_variable'].tolist()

# Se elimina la columna dimension del listado de variables numericas si existe
if columna_dimension in variables_numericas:
    variables_numericas.remove(columna_dimension)

# Definición de funciones para procesamiento masivo por variables
Se construyen funciones para ejecutar en el recorrido del listado de variables (columnas de la tabla de respuestas)

In [19]:
# Función para obtener el promedio de la ciudad
def promedio_ciudad(columna):
    df_sin_na = df_respuestas.dropna(subset=[columna])
    promedio = (df_sin_na[columna] * df_sin_na['FACTOR']).sum() / df_sin_na['FACTOR'].sum()
    return promedio

In [20]:
def agregar_fila_promedio_ciudad(df_respuestas_acumuladas, columna_actual, promedio_ciudad):
    print(f'Agregando fila de promedio ciudad para la variable {columna_actual} con valor {promedio_ciudad}')
    fila_ciudad = pd.DataFrame({
        'variable': [columna_actual],
        'localidad': ['Bogotá D.C.'],
        'promedio_ponderado': [promedio_ciudad],
        'promedio_ciudad': [promedio_ciudad],
        'nivel': ['Ciudad'],
        'min': [promedio_ciudad],
        'max': [promedio_ciudad],
        'rango': [0],
        'rango_relativo': [0],
        'orden': [22]   # Asignar un orden fijo para la ciudad
    })

    df_respuestas_acumuladas = pd.concat([df_respuestas_acumuladas, fila_ciudad], ignore_index=True)
    return df_respuestas_acumuladas


In [21]:
def agregar_promedio_dimension(df_respuestas_acumuladas, columna_actual, promedio_ciudad):
    print(f'→ Procesando variable: {columna_actual}')

    # 1. Validar si la columna existe en df_respuestas
    if columna_actual not in df_respuestas.columns:
        print(f'⚠️  La columna "{columna_actual}" no existe en df_respuestas. Se omite.')
        return df_respuestas_acumuladas

    # 2. Filtrar columnas necesarias y quitar NA
    try:
        df = df_respuestas[[columna_actual, columna_dimension, 'FACTOR']].dropna()
    except Exception as e:
        print(f'⚠️  Error al intentar leer columnas para "{columna_actual}": {e}')
        return df_respuestas_acumuladas

    # 3. Verificar si la columna_actual quedó vacía después del dropna
    if df.empty:
        print(f'⚠️  La columna "{columna_actual}" no tiene datos válidos (todo es NA o se filtró todo). Se omite.')
        return df_respuestas_acumuladas

    # 4. Calcular el promedio ponderado por dimensión
    try:
        df_respuestas_acumuladas_pre = (
            df.groupby(columna_dimension)
              .apply(lambda x: (x[columna_actual] * x['FACTOR']).sum() / x['FACTOR'].sum())
              .reset_index(name='promedio_ponderado')
        )
    except Exception as e:
        print(f'⚠️  Error al calcular promedios para "{columna_actual}": {e}')
        return df_respuestas_acumuladas

    # 5. Renombrar la columna de dimensión
    df_respuestas_acumuladas_pre = df_respuestas_acumuladas_pre.rename(
        columns={columna_dimension: columna_dimension_nombre}
    )

    # Identificar el valor mínimo entre todas las localidades
    valor_min = df_respuestas_acumuladas_pre['promedio_ponderado'].min()
    valor_max = df_respuestas_acumuladas_pre['promedio_ponderado'].max()

    # 6. Agregar metadatos
    df_respuestas_acumuladas_pre['variable'] = columna_actual
    df_respuestas_acumuladas_pre['promedio_ciudad'] = promedio_ciudad
    df_respuestas_acumuladas_pre['nivel'] = 'Localidad'
    df_respuestas_acumuladas_pre['min'] = valor_min
    df_respuestas_acumuladas_pre['max'] = valor_max
    df_respuestas_acumuladas_pre['rango'] = valor_max - valor_min
    df_respuestas_acumuladas_pre['rango_relativo'] = (valor_max - valor_min) / promedio_ciudad if promedio_ciudad != 0 else None

    # 7. Agregar una columna de orden según el valor del promedio ponderado, el valor más alto es el primero
    df_respuestas_acumuladas_pre = df_respuestas_acumuladas_pre.sort_values(by='promedio_ponderado', ascending=False).reset_index(drop=True)
    df_respuestas_acumuladas_pre['orden'] = df_respuestas_acumuladas_pre.index + 1

    # 8. Reordenar columnas de salida
    df_respuestas_acumuladas_pre = df_respuestas_acumuladas_pre[
        ['variable', columna_dimension_nombre, 'promedio_ponderado', 'promedio_ciudad', 'nivel', 'min', 'max', 'rango', 'rango_relativo', 'orden']
    ]

    # 9. Concatenar al acumulado general
    df_respuestas_acumuladas = pd.concat(
        [df_respuestas_acumuladas, df_respuestas_acumuladas_pre],
        ignore_index=True
    )

    print(f'✔️  Agregado correctamente: {columna_actual}')
    return df_respuestas_acumuladas


In [22]:
# Recorrer las variables numéricas y agregar los promedios al df_respuestas_acumuladas
for columna_actual in variables_numericas:
    promedio = promedio_ciudad(columna_actual)
    df_respuestas_acumuladas = agregar_fila_promedio_ciudad(
        df_respuestas_acumuladas,
        columna_actual,
        promedio
    )
    df_respuestas_acumuladas = agregar_promedio_dimension(
        df_respuestas_acumuladas,
        columna_actual,
        promedio
    )

Agregando fila de promedio ciudad para la variable P0003007000 con valor 1.3935906978688801
→ Procesando variable: P0003007000
✔️  Agregado correctamente: P0003007000
Agregando fila de promedio ciudad para la variable P0002001000 con valor 2.797861777928432
→ Procesando variable: P0002001000
✔️  Agregado correctamente: P0002001000
Agregando fila de promedio ciudad para la variable EDAD con valor 40.797647380634075
→ Procesando variable: EDAD
✔️  Agregado correctamente: EDAD
Agregando fila de promedio ciudad para la variable P0100044000 con valor 4.980911288706554
→ Procesando variable: P0100044000
✔️  Agregado correctamente: P0100044000
Agregando fila de promedio ciudad para la variable P0100045000 con valor 42.573655173928216
→ Procesando variable: P0100045000
✔️  Agregado correctamente: P0100045000
Agregando fila de promedio ciudad para la variable P0100046000 con valor 211.41224271059087
→ Procesando variable: P0100046000
✔️  Agregado correctamente: P0100046000
Agregando fila de pro

C:\Users\User\AppData\Local\Temp\ipykernel_6320\973285850.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x[columna_actual] * x['FACTOR']).sum() / x['FACTOR'].sum())
C:\Users\User\AppData\Local\Temp\ipykernel_6320\973285850.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x[columna_actual] * x['FACTOR']).sum() / x['FACTOR'].sum())
C:\Users\User\AppData\Local\Temp

✔️  Agregado correctamente: P0100047000
Agregando fila de promedio ciudad para la variable P0100049000 con valor 529.3538984942925
→ Procesando variable: P0100049000
✔️  Agregado correctamente: P0100049000
Agregando fila de promedio ciudad para la variable P1500007000 con valor 200.58715677424559
→ Procesando variable: P1500007000
✔️  Agregado correctamente: P1500007000
Agregando fila de promedio ciudad para la variable P1500007150_1 con valor 103.80333037590066
→ Procesando variable: P1500007150_1
✔️  Agregado correctamente: P1500007150_1
Agregando fila de promedio ciudad para la variable P1500007160_1 con valor 397.49203406903735
→ Procesando variable: P1500007160_1
✔️  Agregado correctamente: P1500007160_1
Agregando fila de promedio ciudad para la variable P1500007030 con valor 206.31675076524252
→ Procesando variable: P1500007030
✔️  Agregado correctamente: P1500007030
Agregando fila de promedio ciudad para la variable P1500007010 con valor 90.01168963274048
→ Procesando variable: 

C:\Users\User\AppData\Local\Temp\ipykernel_6320\973285850.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x[columna_actual] * x['FACTOR']).sum() / x['FACTOR'].sum())


## Se agregan variables complementarias a la tabla

In [23]:
# Columna con la diferencia entre el promedio de la localidad y el de la ciudad
df_respuestas_acumuladas['diferencia_ciudad'] = df_respuestas_acumuladas['promedio_ponderado'] - df_respuestas_acumuladas['promedio_ciudad']

# Ordenar el dataframe por variable y luego por promedio_ponderado descendente
df_respuestas_acumuladas = df_respuestas_acumuladas.sort_values(by=['variable', 'promedio_ponderado'], ascending=[True, False]).reset_index(drop=True)

# Agregar las columnas enunciado_2, indice_pregunta, variable_nombre desde df_variables
df_respuestas_acumuladas = df_respuestas_acumuladas.merge(
    df_variables[['codigo_variable', 'enunciado_2', 'indice_pregunta', 'variable_nombre', 'unidad_medida']],
    left_on='variable',
    right_on='codigo_variable',
    how='left'
).drop(columns=['codigo_variable'])

# Agregar las columnas desde df_preguntas, vinculadas por indice_pregunta
df_respuestas_acumuladas = df_respuestas_acumuladas.merge(
    df_preguntas[['indice_pregunta', 'enunciado_1', 'etiqueta_1', 'num_seccion']],
    on='indice_pregunta',
    how='left'
)

# Agregar columnas "Código localidad", "Nombre localidad" desde df_localidades, vinculada por "localidad" = "Localidad residencia"
df_respuestas_acumuladas = df_respuestas_acumuladas.merge(
    df_localidades[['Código localidad', 'Nombre localidad', 'Localidad residencia']],
    left_on='localidad',
    right_on='Localidad residencia',
    how='left'
).drop(columns=['Nombre localidad', 'Localidad residencia'])

# Agregar columna df_secciones 'nombre_seccion' vinculada por 'num_seccion'
df_respuestas_acumuladas = df_respuestas_acumuladas.merge(
    df_secciones[['num_seccion', 'nombre_seccion']],
    on='num_seccion',
    how='left'
)


In [24]:
# Guardar el dataframe resultante en un archivo Excel ya exisente, hoja 'analisis_territorial'
resultados_filename = 'm182_datos_acumulados.xlsx'
with pd.ExcelWriter(resultados_filename, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_respuestas_acumuladas.to_excel(writer, sheet_name='resultados_localidad', index=False)
